In [ ]:
grid = dict()
start = ()
with open("day21_input.txt") as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            if char == "#":
                grid[(row, col)] = char
            if char == "S":
                start = (row, col)

MAX_COL = col
MAX_ROW = row

In [ ]:
def neighbours(pos):
    row, col = pos
    for nrow, ncol in [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]:
        if (
            (0 <= nrow <= MAX_ROW)
            and (0 <= ncol <= MAX_COL)
            and ((nrow, ncol) not in grid)
        ):
            yield (nrow, ncol)

# Part 1


In [ ]:
from collections import deque, Counter


def num_reachable(max_steps, neighbour_func=neighbours):
    """Use BFS to find the distance from `start` to all nodes we can reach with a
    certain number of steps."""
    distances = {start: 0}
    queue = deque([start])
    while queue:
        node = queue.popleft()
        if distances[node] > max_steps:
            break
        for neighbour in neighbour_func(node):
            if neighbour not in distances:
                distances[neighbour] = distances[node] + 1
                queue.append(neighbour)

    # The elf can also revisit nodes he has already visited. So we need to count the number
    # of nodes at each distance. At step N, we can reach all nodes at distance N, N-2, N-4, ...
    return sum(Counter(distances.values())[d] for d in range(max_steps, -1, -2))


print(f"Answer: {num_reachable(64)}")

# Part 2


In [ ]:
def neighbours_part2(pos):
    """Neighbours for part 2. The grid is infinite, so we need to wrap around."""
    row, col = pos
    for nrow, ncol in [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]:
        if (nrow % (MAX_ROW + 1), ncol % (MAX_COL + 1)) not in grid:
            yield (nrow, ncol)

The input grid is 131 x 131. From the `S` in the middle, there are 65 steps in a
straight, unblocked line to the edge of the grid in all four directions. Thus, it takes
65 steps to fill the first center-grid.

After filling the first grid in 65 steps, each repetition of the grid will take
additional 131 steps to fill. For each new "layer" of grids filled, the number of
reachable steps will increase quadratically.

Luckily for us, the number of steps in the question is _exactly_ after filling a certain
number of grids, after the initial center grid.:

$$ (26501365 - 65) / 131 = 202300 $$

I guess it is also not a coincidence that $ 202300 = 2023 \times 100 $...

Since we are interested in the number of reachable nodes after `202300` additional grids
are filled, lets see how the number of reachable nodes increase as we fill more grids:


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, 5)  # Number of grids filled
y = [num_reachable(i, neighbours_part2) for i in x * 131 + 65]
coefs = np.polyfit(x, y, 2)
error = np.round(y - np.poly1d(coefs)(x), 5)
# Note that the fit is perfect, so the error is 0
print(f"{error=}")

# Plot the data and the fitted curve
xpred = np.linspace(1, 4, 100)
yhat = np.poly1d(coefs)(xpred)
plt.plot(x, y, "o", xpred, yhat, "-")
plt.xticks(x)
plt.xlabel("Number of surrounding grids filled")
plt.ylabel("Number of reachable nodes")

We get a perfect fit with $$ f(x) = 14860 \, x^2 + 14925 \, x + 14925 $$ where $x$ is
the number of additional 131-grids filled after the initial 65-step center grid.


In [ ]:
# Since we are able to perfectly predict the number of reachable nodes for mulitples of
# 131, we can now use the fitted polynomial to predict the number of reachable nodes:
answer = np.poly1d(coefs)((26501365 - 65) / 131)
print("Answer:", answer)